In [ ]:
#Тут тестил Yolo
from ultralytics import YOLO
import cv2

# Загружаем модель
model = YOLO("yolov8n.pt")  # ← замените на вашу обученную модель на здания!

# Путь к изображению
img_path = #Тут путь к датасету

# Детекция с минимальным порогом уверенности
results = model(img_path, classes=60, conf=0.001) #Тут смотрел именно оригинальную Yolo с таргетом на дома

# Визуализируем результат
for result in results:
    plotted_img = result.plot() 

    # Сохраняем результат в файл
    output_path = "detected_result.jpg"
    cv2.imwrite(output_path, plotted_img)
    print(f"Результат сохранён: {output_path}")

    # Отображаем в окне (если есть GUI)
    cv2.imshow("Detection Result", plotted_img)
    print("Нажмите любую клавишу в окне OpenCV, чтобы закрыть...")
    cv2.waitKey(0)  # Ждём нажатия клавиши
    cv2.destroyAllWindows()

    # Выводим координаты, уверенность и класс
    for box in result.boxes:
        xyxy = box.xyxy.tolist()[0]  # [x1, y1, x2, y2]
        conf = box.conf.item()
        cls = box.cls.item()
        print(f"Box: {xyxy}, Confidence: {conf:.4f}, Class: {cls}")


image 1/1 /home/lanmo/hack_digital_transformation/src/data/ИНС/18-001_gin_building_echd_19.08.25/0a03d321-0a5f-4344-87d4-c2e2c20dd761.jpg: 384x640 (no detections), 237.8ms
Speed: 15.5ms preprocess, 237.8ms inference, 5.3ms postprocess per image at shape (1, 3, 384, 640)
Результат сохранён: detected_result.jpg
Нажмите любую клавишу в окне OpenCV, чтобы закрыть...


In [ ]:
#Даталоадер
from torchvision import transforms
from torch.utils.data import Dataset
from PIL import Image
import pandas as pd
import torch

CLIP_MEAN = [0.48145466, 0.4578275, 0.40821073]
CLIP_STD  = [0.26862954, 0.26130258, 0.27577711]

def build_transforms(is_train = True):
    if is_train:
        return transforms.Compose([
            transforms.Resize(256),
            transforms.RandomResizedCrop(224, scale=(0.6,1.0)),
            transforms.RandomApply([transforms.ColorJitter(0.2,0.2,0.2,0.1)], p=0.7),
            transforms.RandomGrayscale(p=0.1),
            transforms.ToTensor(),
            transforms.Normalize(CLIP_MEAN, CLIP_STD)
        ])
    else:
        return transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(CLIP_MEAN, CLIP_STD)
        ])

class ImgGpsDataset(Dataset):
    def __init__(self, csv_path, split=True):
        self.df = pd.read_excel(csv_path)
        self.tfm = build_transforms(split)
    def __len__(self): return len(self.df)
    def __getitem__(self, i):
        r = self.df.iloc[i]
        path = f'/home/lanmo/hack_digital_transformation/src/data/ИНС/18-001_gin_building_echd_19.08.25/{r['Имя файла']}'
        img = self.tfm(img)
        latlon = torch.tensor([r["latitude"], r["longitude"]], dtype=torch.float32)
        return {"image": img, 
                "latlon": latlon}


In [26]:
data = ImgGpsDataset(r'/home/lanmo/hack_digital_transformation/src/data/ИНС/18-001_gin_building_echd_19.08.25.xlsx')

In [4]:
from geoclip import GeoCLIP

model = GeoCLIP()             
gps_topk, prob_topk = model.predict(r"/home/lanmo/hack_digital_transformation/src/data/ИНС/18-001_gin_building_echd_19.08.25/0a2e94ee-534f-447e-9241-da1c375a474e.jpg", top_k=5)

# Пайплайн локализации по фото (GeoCLIP + региональные данные)

## 3.1 Сбор и подготовка данных (офлайн)

**Что:** собрать пары *(изображение → lat,lon)* под Москву/МО (Mapillary / свои / панорамы), плюс геоконтуры зданий (OSM / кадастр).  
**Зачем:** локальные данные = устойчивость к сезону, ночным сценам, типичной архитектуре.  
**Результат:** датасет *train/val/test* и «галерея» координат региона (сетка 30–80 м или центры building footprints).

---

## 3.2 Создание региональной галереи координат

**Что:** кодируем каждую точку *(lat,lon)* GPS-энкодером GeoCLIP → эмбеддинги; индексируем FAISS (IVF-PQ / HNSW).  
**Зачем:** мгновенный *top-K* поиск кандидатов по эмбеддингу изображения; ограничиваемся регионом → меньше ошибок.  
**Результат:** готовый индекс (файл), к которому обращаемся при инференсе.

---

## 3.3 Дообучение GeoCLIP под Москву/МО

**Что:** берём предобученный GeoCLIP; замораживаем часть *image-backbone*; учим контрастно *(image ↔ gps)* на локальных парах; аугментации под зиму/ночь/дождь.  
**Зачем:** «подсадить» модель на московский доменный сдвиг, поднять точность на радиусах 25–100 м.  
**Результат:** веса модели, которые лучше «читают» московские сцены.

---

## 3.4 Валидация и метрики

**Что:** на *val* считаем геодезическую ошибку и  
`Accuracy@{25,50,100,250} м`, `Median error`, `Recall@K` (для поиска).  
**Зачем:** видеть реальный прогресс «в метрах», не абстрактные баллы.  
**Результат:** отчёт качества; early-stopping / подбор гиперпараметров.

---

## 3.5 Инференс (онлайн)

**Что:** фото → препроцесс → *image-эмбеддинг* → `FAISS.search` по региональной галерее → *top-K* координат.  
**Зачем:** быстро сузить до десятков / сотни лучших гипотез.  
**Результат:** список кандидатов (координаты + расстояние в эмбеддингах).

---

## 3.6 Уточнение (опционально, для высокой точности)

- **Вариант A (кросс-вид):** на *top-K* тайлах спутника делаем переранжирование (SAFA / дескрипторы + SuperGlue + геометрия).  
- **Вариант B (моно-вид):** локальный рефайн вокруг кандидатов (NCC / гомография / мульти-масштаб).  

**Зачем:** поднять точность до десятков метров и отсеять «ложных близнецов».  
**Результат:** 1–3 лучших гипотез с более надёжными баллами.

---

## 3.7 Калибровка уверенности

**Что:** по *val* обучаем простую калибровку (temperature / логистическая регрессия) для *P(ошибка ≤ r)*.  
**Зачем:** чтобы система возвращала осмысленную вероятность, а не «сырые» скоринги.  
**Результат:** вместе с координатой отдаём вероятности для *r ∈ {25,50,100,250} м*.

---

## 3.8 Пост-обработка («что сделать дальше»)

- **Reverse-geocoding:** адрес (улица / дом / индекс).  
- **Кадастр / контур:** привязка к building footprint / кадастровому номеру.  
- **Бизнес-логика:** маршрутизация, проверка объекта, карточка в CRM и т.п.  

**Результат:** прикладная ценность — координаты превращаются в понятные сущности / действия.

---

## 3.9 Мониторинг и поддержка

**Что:** логирование ошибок в метрах, теплокарты по районам, детекция деградации (сезоны / новая застройка).  
**Зачем:** качество не «плывёт», можно быстро дообучить на новой порции данных.  
**Результат:** стабильная работа в проде.
